Recogida de datos

In [1]:
import sqlite3

def guardar(tabla, url, informacion):
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()
    
        # Verificar si ya existe el registro antes de la inserción
        cursor.execute(f"SELECT COUNT(*) FROM {tabla} WHERE url = ? AND informacion = ?", (url, informacion))
        registroExistente = cursor.fetchone()[0]

        if not registroExistente:
            # Evitar posibles problemas de SQL injection utilizando parámetros de la consulta
            cursor.execute(f"INSERT INTO {tabla} (url, informacion) VALUES (?, ?)", (url, informacion))
            print(f"Guardado en {tabla}: {url}, {informacion}")
            conexion.commit()
        else:
            print(f"Registro existente en {tabla}: {url}, {informacion}")

            
    conexion.close()

In [2]:
#CREAMOS UN ARCHIVO PARA GUARDAR LA URL:
def listaUrls(urlNueva):
    # Verificamos si la URL ya existe en el archivo

    try:
        # Verificamos si el archivo existe
        archivo = open("urls.txt", 'r')
        urlsExistentes = archivo.read().splitlines()
        archivo.close()
    except FileNotFoundError:
        # Si el archivo no existe, lo creamos
        archivo = open("urls.txt", 'w')
        urlsExistentes = []
        archivo.close()


    if urlNueva not in urlsExistentes:
        # Si la URL no existe, añadir al final del archivo
           
        archivo = open("urls.txt", 'a')
        archivo.write(f"{urlNueva}\n")
        print(f"URL añadida: {urlNueva}")
        archivo.close()
        return True
            
    else:
        print(f"La URL ya existe en el archivo: {urlNueva}")
        return False



In [3]:
def raspado(urls, etiquetas):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urlparse

    for url in urls: #Verificamos si hay alguna url inválida
        parsed_url = urlparse(url)
        if not (parsed_url.scheme and parsed_url.netloc):
            print(f"URL inválida: {url}")
            continue
        if listaUrls(url):
            contenido = requests.get(url)
            sopa = BeautifulSoup(contenido.text, 'html.parser')

            for etiqueta in etiquetas:
                elementos = sopa.find_all(etiqueta)
                
                for elemento in elementos:
                    if etiqueta == 'a':
                        # Si la etiqueta es 'a' (enlace), obtener la URL del atributo 'href'
                        informacion = elemento.get('href')
                        if informacion is not None and not informacion.startswith(('https:', '//',"http:")):
                            continue
                    elif etiqueta == 'img':
                        # Si la etiqueta es 'img' (imagen), obtener la URL del atributo 'src'
                        informacion = elemento.get('src')
                        # Agregar condición para verificar si la URL comienza con 'https' o '//'
                        if informacion is not None and not informacion.startswith(('https:', '//',"http:")):
                            continue  # Salta a la siguiente iteración si no cumple con la condición
                    else:
                        # En otros casos, obtener el texto del elemento
                        informacion = elemento.text.strip()

                    # Guardar en la base de datos
                    guardar(etiqueta, url, informacion)


In [4]:
listaUrl =  ["https://es.wikipedia.org/wiki/Bulldog",
             "https://www.hillspet.es/dog-care/dog-breeds/english-bulldog",
             "https://www.zooplus.es/magazine/perros/razas-de-perro/bulldog-ingles",
             "https://gudog.com/blog/todo-lo-que-tienes-que-saber-antes-de-tener-un-bulldog-ingles",
             "https://piensosloboazul.com/bulldog-ingles-origen-caracteristicas-alimentacion-y-mucho-mas/",
             "https://www.tiendanimal.es/articulos/bulldog-ingles-como-cuidarlo/"
]
listaEtiquetas = ["h1","p","a","img",]

In [5]:
raspado(listaUrl, listaEtiquetas)

La URL ya existe en el archivo: https://es.wikipedia.org/wiki/Bulldog
La URL ya existe en el archivo: https://www.hillspet.es/dog-care/dog-breeds/english-bulldog
La URL ya existe en el archivo: https://www.zooplus.es/magazine/perros/razas-de-perro/bulldog-ingles
La URL ya existe en el archivo: https://gudog.com/blog/todo-lo-que-tienes-que-saber-antes-de-tener-un-bulldog-ingles
La URL ya existe en el archivo: https://piensosloboazul.com/bulldog-ingles-origen-caracteristicas-alimentacion-y-mucho-mas/
La URL ya existe en el archivo: https://www.tiendanimal.es/articulos/bulldog-ingles-como-cuidarlo/


Creación de contenido

In [6]:
#Vamos a realizar una funcion que escoja los párrafos en los que mas veces se repita la palabra indicada
import re

def obtenerCincoConMasCoincidencias(lista, palabra):
    # Definir una expresión regular para buscar la palabra clave ignorando mayúsculas y minusculas
    patron = re.compile(re.escape(palabra), flags=re.IGNORECASE)

    # Creamos otra función para contar las veces que aparece las palabra clave en una cadena (ignorando mayúsculas y minusculas)
    def veces(cadena):
        return len(re.findall(patron, cadena.lower()))

    #Ordenamos los str por la cantidad de veces que se repite palabra
    listaOrdenada = sorted(lista, key=lambda x: veces(x), reverse=True)

    # Tomar las tres cadenas con más ocurrencias
    top5 = listaOrdenada[:5]
    return top5

In [7]:
import sqlite3
from unidecode import unidecode  


def generarContenido(palabra):
    #Vamos a guardar la palabra sin tildes para buscar las imágenes y los enlaces
    palabraSinTilde = unidecode(palabra)
    # Establecemos conexión con la BBDD
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()


        # Buscamos información en la BBDD
        # Buscamos un título
        cursor.execute("SELECT * FROM h1 WHERE informacion LIKE ?",
                       ('%' + palabra + '%',))

        h1 = cursor.fetchall()


        # Buscar los parrafos
        cursor.execute("SELECT * FROM p WHERE informacion LIKE ?",('%' + palabra + '%',))

        p = cursor.fetchall()
    
        # Escoger el mejor contenido
        parrafos = []
        for row in p:
            parrafo = row[2]
            parrafos.append(parrafo)


        # Busca imágenes, vamos a quitar los espacios para que salgan las palabras

        variantes = [palabraSinTilde.replace(' ', ''), palabraSinTilde.replace(
            ' ', '-'), palabraSinTilde.replace(' ', '_')]
        img = None
        for variante in variantes:
            cursor.execute(
                "SELECT * FROM img WHERE informacion LIKE ?", ('%' + variante + '%',))
            img = cursor.fetchall()
            if img:
                # Si se encuentramos datos, salimos del bucle
                break   
        # Hacemos otro con las img:
        imagenes = []
        for row in img:
            imagen = row[2]

            if not imagen.startswith(("http:", "https:")):

                # Si no comienza con "http: o https:" lo agregamos

                imagen = "https:" + imagen

            imagenes.append(imagen)

        # Buscamos ENLACES relacionados
        variantes = [palabraSinTilde.replace(' ', ''), palabraSinTilde.replace(
            ' ', '-'), palabraSinTilde.replace(' ', '_')]
        a = None
        for variante in variantes:
            cursor.execute(
                "SELECT * FROM a WHERE informacion LIKE ?", ('%' + variante + '%',))
            a = cursor.fetchall()
            if a:
                # Si se encuentramos datos, salimos del bucle
                break
        enlaces = []
        for row in a:
            enlace = row[2]
            if not enlace.startswith(("/w", "/wiki")):
                if not enlace.startswith(("http:", "https:")):
                    # Si no comienza con "http: o https:" lo agregamos
                    enlace = "https:" + enlace

                enlaces.append(enlace)


    top5Parrafo = obtenerCincoConMasCoincidencias(parrafos, palabra)

    top5Imagen = obtenerCincoConMasCoincidencias(imagenes, palabraSinTilde)


    # Generar un archivo HTML con el contenido obtenido

    with open('plantilla.html', 'r', encoding='utf-8') as template_file:

        template = template_file.read()


        # titulo

        template = template.replace(
            '<!--Título-->', f'{h1[0][2]}' if h1 else palabra)

        # enlace
        template = template.replace(
            'Enlace a la web', f'{h1[0][1]}' if h1 else '')

        # parrafo
        template = template.replace(
            '<!--Texto_Principal-->', f'{top5Parrafo[0]}' if top5Parrafo else '')


        # Enlace al párrafo ###########Pendiente de mejorar
        template = template.replace(
            '+info', f'{p[0][1]}' if top5Parrafo else '')

        # Imagen de logo
        template = template.replace(
            'Imagen logo', f'{top5Imagen[0]}' if top5Imagen else 'noImage.jpg')

        # imagen principal
        template = template.replace('Imagen principal', f'{top5Imagen[1]}' if top5Imagen and len(top5Imagen) > 1 else 'noImage.jpg')
        # Texto carrusel
        template = template.replace(
            '<!--Texto_carrusel-->', f'{top5Parrafo[1]}' if top5Parrafo else '')

        # Imagenes carrusel
        template = template.replace(
            'Imagen1', f'{top5Imagen[2]}' if top5Imagen and len(top5Imagen) > 2 else 'noImage.jpg')
        template = template.replace(
            'Imagen2', f'{top5Imagen[3]}' if top5Imagen and len(top5Imagen) > 3 else 'noImage.jpg')

        template = template.replace(
            'Imagen3', f'{top5Imagen[4]}' if top5Imagen and len(top5Imagen) > 4 else 'noImage.jpg')


        # Lista de enlaces relacionados
        template = template.replace('listaUno', f'{enlaces[0]}' if enlaces and len(enlaces) > 0 else '')
        template = template.replace('listaDos', f'{enlaces[1]}' if len(enlaces) > 1 else '')
        template = template.replace('listaTres', f'{enlaces[2]}' if len(enlaces) > 2 else '')
        template = template.replace('listaCuatro', f'{enlaces[3]}' if len(enlaces) > 3 else '')

    with open('output.html', 'w', encoding='utf-8') as output_file:

        output_file.write(template)

In [9]:
generarContenido('bulldog francés')